In [74]:
from datasets import load_dataset, concatenate_datasets, Dataset
import pandas as pd
import json
import subprocess
import re

In [63]:
final_dataset = []

In [64]:
map_sentiment = {'POS': 'positive', 'NEG':'negative', 'NEU': 'neutral'}
def aspect_based(examples):
    domains = examples['domain']
    texts = examples['sentence']
    sources = examples['source']

    target_sources = []
    target_texts = []
    target_aspects = []
    target_sentiments = []
    for idx, aspects in enumerate(examples['triples']):
        for aspect in eval(aspects):
            a, b, c = aspect
            if a == -1:
                a = domains[idx]
            sentiment = map_sentiment[c]
            target_sources.append("SilvioLima/"+sources[idx])
            target_texts.append(texts[idx])
            target_aspects.append(a)
            target_sentiments.append(sentiment)
    return {'source': target_sources,
            'text': target_texts,
            'aspect': target_aspects,
            'label_text': target_sentiments}

ds = load_dataset("SilvioLima/absa")
f = []
for split in ['test', 'valid', 'train']:
    df = ds[split].map(aspect_based, batched = True, remove_columns= ds[split].column_names)
    df = df.to_pandas()
    df['split'] = split
    f.append(df)

final_df = pd.concat(f)
final_dataset.append(final_df)
final_df


,source,text,aspect,label_text,split
0,SilvioLima/DMASTE,Wow I was totally hyped to try this shower gel...,bottle,negative,test
1,SilvioLima/DMASTE,Wow I was totally hyped to try this shower gel...,package,negative,test
2,SilvioLima/DMASTE,Wow I was totally hyped to try this shower gel...,beauty,negative,test
3,SilvioLima/SemEval-16,I may not be a sushi guru but I can tell you t...,food,negative,test
4,SilvioLima/DMASTE,Wonderful cot . I just spent a week at scout c...,cot,positive,test
...,...,...,...,...,...
30815,SilvioLima/DMASTE,Wish I would have ordered earlier . This makes...,pet,positive,train
30816,SilvioLima/DMASTE,Wish I would have ordered earlier . This makes...,works,positive,train
30817,SilvioLima/SemEval-14,There are much better places in NY with better...,prices,negative,train
30818,SilvioLima/SemEval-14,"Not the greatest sushi place , but excellent f...",sushi place,negative,train


In [65]:
!git clone https://github.com/siat-nlp/MAMS-for-ABSA/

fatal: destination path 'MAMS-for-ABSA' already exists and is not an empty directory.


In [66]:
from xml.etree.ElementTree import parse
def parse_sentence_term(path, lowercase=False):
    tree = parse(path)
    sentences = tree.getroot()
    data = []
    for sentence in sentences:
        text = sentence.find('text')
        if text is None:
            continue
        text = text.text
        if lowercase:
            text = text.lower()
        aspectTerms = sentence.find('aspectTerms')
        if aspectTerms is None:
            continue
        for aspectTerm in aspectTerms:
            term = aspectTerm.get('term')
            if lowercase:
                term = term.lower()
            polarity = aspectTerm.get('polarity')
            piece = [text,term,polarity]
            data.append(piece)
    return data

def parse_sentence_category(path, lowercase=False):
    tree = parse(path)
    sentences = tree.getroot()
    data = []
    for sentence in sentences:
        text = sentence.find('text')
        if text is None:
            continue
        text = text.text
        if lowercase:
            text = text.lower()
        aspectCategories = sentence.find('aspectCategories')
        if aspectCategories is None:
            continue
        for aspectCategory in aspectCategories:
            category = aspectCategory.get('category')
            polarity = aspectCategory.get('polarity')
            piece = [text,category,polarity]
            data.append(piece)
    return data

f = []
for split in ['test', 'val', 'train']:
    df = pd.DataFrame(parse_sentence_category(f"MAMS-for-ABSA/data/MAMS-ACSA/raw/{split}.xml"), columns = ['text','aspect','label_text'])
    df['source'] = "siat-nlp/MAMS-for-ABSA/MAMS-ACSA"
    df['split'] = split
    f.append(df)

    df = pd.DataFrame(parse_sentence_term(f"MAMS-for-ABSA/data/MAMS-ATSA/raw/{split}.xml"), columns = ['text','aspect','label_text'])
    df['source'] = "siat-nlp/MAMS-for-ABSA/MAMS-ATSA"
    df['split'] = split
    f.append(df)

final_df_2 = pd.concat(f)
final_dataset.append(final_df_2)
final_df_2

,text,aspect,label_text,source,split
0,"We went again and sat at the bar this time, I ...",place,neutral,siat-nlp/MAMS-for-ABSA/MAMS-ACSA,test
1,"We went again and sat at the bar this time, I ...",food,negative,siat-nlp/MAMS-for-ABSA/MAMS-ACSA,test
2,"The food was good, but it's not worth the wait...",food,positive,siat-nlp/MAMS-for-ABSA/MAMS-ACSA,test
3,"The food was good, but it's not worth the wait...",service,negative,siat-nlp/MAMS-for-ABSA/MAMS-ACSA,test
4,Waiter took our drink order and then we didn't...,staff,negative,siat-nlp/MAMS-for-ABSA/MAMS-ACSA,test
...,...,...,...,...,...
11181,The menu sounded good but the grilled eggplant...,grilled eggplant roll,negative,siat-nlp/MAMS-for-ABSA/MAMS-ATSA,train
11182,Service is coddling and correct and there's no...,Service,positive,siat-nlp/MAMS-for-ABSA/MAMS-ATSA,train
11183,Service is coddling and correct and there's no...,cheese,neutral,siat-nlp/MAMS-for-ABSA/MAMS-ATSA,train
11184,USC has a cold smoker and smoked the avocado i...,avocado,neutral,siat-nlp/MAMS-for-ABSA/MAMS-ATSA,train


In [67]:
! git clone https://github.com/songyouwei/ABSA-PyTorch

fatal: destination path 'ABSA-PyTorch' already exists and is not an empty directory.


In [68]:
polarity_mapping = {"0": "neutral", "1":"positive", "-1":"negative"}
def parse_raw_file(pathname):
    with open(pathname, "r") as f:
        result = []
        c = 0
        for line in f.readlines():
            if c%3 == 0:
                sentence = line.replace("\n","")
            elif c%3 == 1:
                aspect = line.replace("\n","")
            elif c%3 == 2:
                polarity = line.replace("\n","")
                result.append([sentence.replace("$T$",aspect), aspect, polarity_mapping[polarity]])
            c +=1
    return pd.DataFrame(result, columns = ["text",'aspect','label_text'])
f = []
for split in ['train','test']:
    df = parse_raw_file(f"ABSA-PyTorch/datasets/acl-14-short-data/{split}.raw")
    df['source'] = "songyouwei/ABSA-PyTorch"
    df['split'] = split
    f.append(df)

final_df_3 = pd.concat(f)
final_dataset.append(final_df_3)
final_df_3

,text,aspect,label_text,source,split
0,"i agree about arafat . i mean , shit , they ev...",jimmy carter,negative,songyouwei/ABSA-PyTorch,train
1,musicmonday britney spears - lucky do you reme...,britney spears,positive,songyouwei/ABSA-PyTorch,train
2,wtf ? hilary swank is coming to my school toda...,hilary swank,neutral,songyouwei/ABSA-PyTorch,train
3,my 3-year-old was amazed yesterday to find tha...,wii,neutral,songyouwei/ABSA-PyTorch,train
4,God damn . That Sony remote for google is fuck...,google,negative,songyouwei/ABSA-PyTorch,train
...,...,...,...,...,...
687,not a goddamn thing . you got me listening to ...,britney spears,neutral,songyouwei/ABSA-PyTorch,test
688,barack obama made the bullies stop picking on ...,barack obama,negative,songyouwei/ABSA-PyTorch,test
689,apple has better windows 7 support than sony d...,windows 7,positive,songyouwei/ABSA-PyTorch,test
690,"2 hours later , no longer seeing confused russ...",steve jobs,neutral,songyouwei/ABSA-PyTorch,test


In [69]:
ds = load_dataset("eastwind/semeval-2016-absa-reviews-english-translated-stanford-alpaca")
f = []
for split in ['test', 'train']:
    df = ds[split].to_pandas()
    df['source'] = "eastwind/semeval-2016-absa-reviews-english-translated-stanford-alpaca"
    df['split'] = split
    f.append(df.drop(columns = ['prompt','category']).rename(columns = {'sentence':'text','sentiment':'label_text'}))

final_df_4 = pd.concat(f)
final_dataset.append(final_df_4)
final_df_4


,text,aspect,label_text,source,split
0,beautiful hotel beautiful hotel and breakfast ...,views,positive,eastwind/semeval-2016-absa-reviews-english-tra...,test
1,some of the receptionists are unfriendly and t...,receptionists,negative,eastwind/semeval-2016-absa-reviews-english-tra...,test
2,some of the receptionists are unfriendly and t...,cleanliness,positive,eastwind/semeval-2016-absa-reviews-english-tra...,test
3,some of the receptionists are unfriendly and t...,rooms,negative,eastwind/semeval-2016-absa-reviews-english-tra...,test
4,nice hotel.,hotel,positive,eastwind/semeval-2016-absa-reviews-english-tra...,test
...,...,...,...,...,...
9530,lots of tea or coffee with homemade cakes and ...,amenities,positive,eastwind/semeval-2016-absa-reviews-english-tra...,train
9531,lots of tea or coffee with homemade cakes and ...,amenities,positive,eastwind/semeval-2016-absa-reviews-english-tra...,train
9532,the best breakfast in a small hotel with a lot...,meals,positive,eastwind/semeval-2016-absa-reviews-english-tra...,train
9533,the best breakfast in a small hotel with a lot...,hotel,positive,eastwind/semeval-2016-absa-reviews-english-tra...,train


In [70]:
def preperocess_ds(examples):
    target_content, target_aspect, target_sentiment = [], [] , []
    for content, aspect, sentiment in zip(examples['content'],examples['aspect'],examples['sentiment']):
        aspect = [t.strip() for t in aspect.split(',')]
        sentiment = [t.lower().strip() for t in sentiment.replace("(initially positive, later negative)","").split(',')]
        content = [content for i in aspect]
        if len(sentiment) == len(aspect):
            target_content.extend(content)
            target_aspect.extend(aspect)
            target_sentiment.extend(sentiment)
    return {'text': target_content,
            'aspect': target_aspect,
            'label_text': target_sentiment}

ds = load_dataset('Alpaca69B/reviews_appstore_all_absa')
f = []
for split in ['test', 'validation', 'train']:
    df = ds[split].map(preperocess_ds, batched = True, remove_columns= ds[split].column_names)
    df = df.to_pandas()
    df['source'] = "Alpaca69B/reviews_appstore_all_absa"
    df['split'] = split
    f.append(df)

final_df_5 = pd.concat(f)
final_dataset.append(final_df_5)
final_df_5


,aspect,text,label_text,source,split
0,chat,I feel extremely disappointed about this app (...,negative,Alpaca69B/reviews_appstore_all_absa,test
1,conversation,I feel extremely disappointed about this app (...,negative,Alpaca69B/reviews_appstore_all_absa,test
2,message,I feel extremely disappointed about this app (...,negative,Alpaca69B/reviews_appstore_all_absa,test
3,photo,I feel extremely disappointed about this app (...,negative,Alpaca69B/reviews_appstore_all_absa,test
4,video,I feel extremely disappointed about this app (...,negative,Alpaca69B/reviews_appstore_all_absa,test
...,...,...,...,...,...
7360,Clan Wars,"Okay let me get this straight, I have been con...",negative,Alpaca69B/reviews_appstore_all_absa,train
7361,Fun,"Okay let me get this straight, I have been con...",positive,Alpaca69B/reviews_appstore_all_absa,train
7362,Entertainment,"Okay let me get this straight, I have been con...",positive,Alpaca69B/reviews_appstore_all_absa,train
7363,Upgrading,"Okay let me get this straight, I have been con...",positive,Alpaca69B/reviews_appstore_all_absa,train


In [71]:
def preperocess_ds(examples):
    target_content, target_aspect, target_sentiment = [], [] , []
    for content, quad in zip(examples['comment'],examples['quad']):
        for aspect, sentiment in quad:
            target_content.append(content)
            target_aspect.append(aspect)
            target_sentiment.append(sentiment)
    return {'text': target_content,
            'aspect': target_aspect,
            'label_text': target_sentiment}

ds = load_dataset("jordiclive/OATS-ABSA")
f = []
for split in ['test', 'validation', 'train']:
    df = ds[split].map(preperocess_ds, batched = True, remove_columns= ds[split].column_names)
    df = df.to_pandas()
    df['source'] = "jordiclive/OATS-ABSA"
    df['split'] = split
    f.append(df)

final_df_6 = pd.concat(f)
final_dataset.append(final_df_6)
final_df_6


,text,aspect,label_text,source,split
0,"Don't Believe the Reviews - Read This First ""T...",service general,positive,jordiclive/OATS-ABSA,test
1,"Don't Believe the Reviews - Read This First ""T...",rooms design_features,negative,jordiclive/OATS-ABSA,test
2,Fabulous room and views! You definitely get wh...,rooms general,positive,jordiclive/OATS-ABSA,test
3,Fabulous room and views! You definitely get wh...,hotel general,positive,jordiclive/OATS-ABSA,test
4,Fabulous room and views! You definitely get wh...,rooms design_features,positive,jordiclive/OATS-ABSA,test
...,...,...,...,...,...
13729,I feel robbed . I did not get any feedback on ...,faculty response,negative,jordiclive/OATS-ABSA,train
13730,I feel robbed . I did not get any feedback on ...,course general,negative,jordiclive/OATS-ABSA,train
13731,"Excellent class , and an even better interface...",course general,positive,jordiclive/OATS-ABSA,train
13732,"Excellent class , and an even better interface...",faculty comprehensiveness,positive,jordiclive/OATS-ABSA,train


In [72]:

def preperocess_ds(examples):
    target_content, target_aspect, target_sentiment = [], [] , []
    for content, aspect, sentiment in zip(examples['text'],examples['ac'],examples['label']):
        content = eval(content)[0]
        target_content.append(content)
        target_aspect.append(aspect)
        target_sentiment.append(sentiment)
    return {'text': target_content,
            'aspect': target_aspect,
            'label_text': target_sentiment}


ds= load_dataset("JaquanTW/fewshot-absaquad")
f = []
for split in ['test', 'validation', 'train']:
    df = ds[split].map(preperocess_ds, batched = True, remove_columns= ds[split].column_names)
    df = df.to_pandas()
    df['source'] = "JaquanTW/fewshot-absaquad"
    df['split'] = split
    f.append(df)

final_df_7 = pd.concat(f)
final_dataset.append(final_df_7)
final_df_7
            

,text,aspect,label_text,source,split
0,Gross food – Wow -,food quality,negative,JaquanTW/fewshot-absaquad,test
1,Great place to relax and enjoy your dinner,ambience general,positive,JaquanTW/fewshot-absaquad,test
2,The decor however seems to be the distraction ...,ambience general,neutral,JaquanTW/fewshot-absaquad,test
3,The decor however seems to be the distraction ...,food quality,negative,JaquanTW/fewshot-absaquad,test
4,The decor however seems to be the distraction ...,food prices,negative,JaquanTW/fewshot-absaquad,test
...,...,...,...,...,...
2620,"The food was excellent as well as service , ho...",service general,positive,JaquanTW/fewshot-absaquad,train
2621,"The food was excellent as well as service , ho...",restaurant general,negative,JaquanTW/fewshot-absaquad,train
2622,"Their sake list was extensive , but we were lo...",drinks style_options,positive,JaquanTW/fewshot-absaquad,train
2623,The in-house lady DJ on Saturday nights has ou...,ambience general,positive,JaquanTW/fewshot-absaquad,train


In [73]:
dataset = pd.concat(final_dataset)
dataset

,source,text,aspect,label_text,split
0,SilvioLima/DMASTE,Wow I was totally hyped to try this shower gel...,bottle,negative,test
1,SilvioLima/DMASTE,Wow I was totally hyped to try this shower gel...,package,negative,test
2,SilvioLima/DMASTE,Wow I was totally hyped to try this shower gel...,beauty,negative,test
3,SilvioLima/SemEval-16,I may not be a sushi guru but I can tell you t...,food,negative,test
4,SilvioLima/DMASTE,Wonderful cot . I just spent a week at scout c...,cot,positive,test
...,...,...,...,...,...
2620,JaquanTW/fewshot-absaquad,"The food was excellent as well as service , ho...",service general,positive,train
2621,JaquanTW/fewshot-absaquad,"The food was excellent as well as service , ho...",restaurant general,negative,train
2622,JaquanTW/fewshot-absaquad,"Their sake list was extensive , but we were lo...",drinks style_options,positive,train
2623,JaquanTW/fewshot-absaquad,The in-house lady DJ on Saturday nights has ou...,ambience general,positive,train


In [77]:
dataset = Dataset.from_pandas(dataset.reset_index(drop=True))
dataset.push_to_hub("carant-ai/compiled-absa-english")

Uploading the dataset shards: 100%|██████████| 1/1 [00:01<00:00,  1.06s/it]


CommitInfo(commit_url='https://huggingface.co/datasets/carant-ai/compiled-absa-english/commit/f213b1efc48b6a17953204573a52694c6e6d9b2e', commit_message='Upload dataset', commit_description='', oid='f213b1efc48b6a17953204573a52694c6e6d9b2e', pr_url=None, pr_revision=None, pr_num=None)